## Загружаю нужные пакеты

In [1]:
pip install transformers

## Импортирую нужные библиотеки

In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
from torch.utils.data import DataLoader, Dataset

from transformers import AdamW, get_linear_schedule_with_warmup, AutoModelForSequenceClassification, AutoTokenizer

## Исследую данные

In [3]:
df = pd.read_csv('/content/meatinfo.csv', sep=';', on_bad_lines='skip')

In [4]:
df.info()
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17893 entries, 0 to 17892
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    17893 non-null  object
 1   mtype   17892 non-null  object
dtypes: object(2)
memory usage: 279.7+ KB


,text,mtype
0,12 частей баранина 12 частей баранина,Баранина
1,"Баранина, 12 частей, зам. цена 260 руб.",Баранина
2,"Баранина, 12 частей, зам. цена 315 руб.",Баранина
3,"Баранина, 12 частей, охл.",Баранина
4,"Баранина, 12 частей, охл. цена 220 руб.",Баранина


## Посмотрю на уникальные значения столбца 'mtype'

In [5]:
df['mtype'].unique()

array(['Баранина', 'Ягнятина', 'Индейка', 'Говядина', 'Свинина', 'Кура',
       'Цыпленок', 'Гусь', 'Буйволятина', 'Оленина', 'Конина', 'Телятина',
       '125р.', 'Кролик', 'Утка', 'Куропатка', 'Парагвай',
       'Говядина, полутуши, 1 категория,  охл., Россия, подвес, В наличии, 10 тонн, 270 руб. кг',
       'Перепел', 'Глухарь', 'Страус', nan, 'Заяц', 'Кенгуру', 'Изюбр',
       'Кабан', '295,00 руб|кг', 'Коза', 'Косуля',
       ' Лопаточная часть (Chuck) буйвол ', 'Лось', 'Марал',
       'Медвежатина', 'Бобер', 'Цесарка', 'Нутрия', 'Feb-20', 'Mar-20',
       '(OFFAL EXP №4407 Аргентина)', 'OFFAL EXP №4407 Аргентина',
       'индейка', 'свиниеа', 'утка', 'цыпленок', 'свинина', 'Рябчик',
       'Тетерев', 'говядина', 'Фазан', 'Як'], dtype=object)

Видно, что некоторые уникальные значения дублируют друг друга. Приведу данные в порядок.

In [6]:
df.loc[df['mtype'] == 'Говядина, полутуши, 1 категория,  охл., Россия, подвес, В наличии, 10 тонн, 270 руб. кг', 'mtype'] = 'Говядина'
df.loc[df['mtype'] == ' Лопаточная часть (Chuck) буйвол ', 'mtype'] = 'Буйволятина'
df.loc[df['mtype'] == 'индейка', 'mtype'] = 'Индейка'
df.loc[df['mtype'] == 'свиниеа', 'mtype'] = 'Свинина'
df.loc[df['mtype'] == 'утка', 'mtype'] = 'Утка'
df.loc[df['mtype'] == 'цыпленок', 'mtype'] = 'Цыпленок'
df.loc[df['mtype'] == 'свинина', 'mtype'] = 'Свинина'
df.loc[df['mtype'] == 'говядина', 'mtype'] = 'Говядина'

In [7]:
df['mtype'].unique()

array(['Баранина', 'Ягнятина', 'Индейка', 'Говядина', 'Свинина', 'Кура',
       'Цыпленок', 'Гусь', 'Буйволятина', 'Оленина', 'Конина', 'Телятина',
       '125р.', 'Кролик', 'Утка', 'Куропатка', 'Парагвай', 'Перепел',
       'Глухарь', 'Страус', nan, 'Заяц', 'Кенгуру', 'Изюбр', 'Кабан',
       '295,00 руб|кг', 'Коза', 'Косуля', 'Лось', 'Марал', 'Медвежатина',
       'Бобер', 'Цесарка', 'Нутрия', 'Feb-20', 'Mar-20',
       '(OFFAL EXP №4407 Аргентина)', 'OFFAL EXP №4407 Аргентина',
       'Рябчик', 'Тетерев', 'Фазан', 'Як'], dtype=object)

Посмотрю на странные уникальные значения.

Проверю значение '125р.'.

In [8]:
df.loc[df['mtype'] == '125р.']

,text,mtype
1090,Голень куриная с/м монолит Гост Голень курина...,125р.


In [9]:
df.loc[df['mtype'] == '125р.', 'mtype'] = 'Кура'

Проверю значение 'Парагвай'.

In [10]:
df.loc[df['mtype'] == 'Парагвай']

,text,mtype
4773,"Говядина, огузок, зам. Говядина,б/к огузок",Парагвай
4822,"Говядина, огузок, зам. цена 420руб. Говядина,б...",Парагвай


In [11]:
df.loc[df['mtype'] == 'Парагвай', 'mtype'] = 'Говядина'

Проверю значение nan.

In [12]:
df.loc[df['mtype'].isna() == True]

,text,mtype
7904,Филе окорочка бк бк Филе окорочка куриного б/...,NaN


In [13]:
df.loc[df['mtype'].isna() == True, 'mtype'] = 'Кура'

Проверю значение '295,00 руб|кг'.

In [14]:
df.loc[df['mtype'] == '295,00 руб|кг']

,text,mtype
10339,Карбонад свиной б/к гофра 20-23 кг Карбонад св...,"295,00 руб|кг"


In [15]:
df.loc[df['mtype'] == '295,00 руб|кг', 'mtype'] = 'Свинина'

Проверю значение 'Feb-20'.

In [16]:
df.loc[df['mtype'] == 'Feb-20']

,text,mtype
14793,окорочка Окорочка куриные,Feb-20
15460,печень куринная Печень куриная,Feb-20


In [17]:
df.loc[df['mtype'] == 'Feb-20', 'mtype'] = 'Кура'

Проверю значение 'Mar-20'.

In [18]:
df.loc[df['mtype'] == 'Mar-20']

,text,mtype
14794,окорочка Окорочка куриные б/х,Mar-20


In [19]:
df.loc[df['mtype'] == 'Mar-20', 'mtype'] = 'Кура'

Проверю значение '(OFFAL EXP №4407 Аргентина)'.

In [20]:
df.loc[df['mtype'] == '(OFFAL EXP №4407 Аргентина)']

,text,mtype
15182,Печень говяжья Аргентина зам Печень говяжья...,(OFFAL EXP №4407 Аргентина)


In [21]:
df.loc[df['mtype'] == '(OFFAL EXP №4407 Аргентина)', 'mtype'] = 'Говядина'

Проверю значение 'OFFAL EXP №4407 Аргентина'.

In [22]:
df.loc[df['mtype'] == 'OFFAL EXP №4407 Аргентина']

,text,mtype
15280,"Печень говяжья Аргентина ,ЗАМ Печень говяжья ...",OFFAL EXP №4407 Аргентина


In [23]:
df.loc[df['mtype'] == 'OFFAL EXP №4407 Аргентина', 'mtype'] = 'Говядина'

Проверю уникальные значения столюца 'mtype'.

In [24]:
df['mtype'].unique()

array(['Баранина', 'Ягнятина', 'Индейка', 'Говядина', 'Свинина', 'Кура',
       'Цыпленок', 'Гусь', 'Буйволятина', 'Оленина', 'Конина', 'Телятина',
       'Кролик', 'Утка', 'Куропатка', 'Перепел', 'Глухарь', 'Страус',
       'Заяц', 'Кенгуру', 'Изюбр', 'Кабан', 'Коза', 'Косуля', 'Лось',
       'Марал', 'Медвежатина', 'Бобер', 'Цесарка', 'Нутрия', 'Рябчик',
       'Тетерев', 'Фазан', 'Як'], dtype=object)

## Обработка пропусков

In [25]:
df.isna().sum()

text     0
mtype    0
dtype: int64

Пропусков обнаружено не было.

## Обработка дубликатов

In [26]:
df.duplicated().sum()

4

In [27]:
df.loc[df.duplicated() == True]

,text,mtype
5689,"Говядина 1 кат. П/т охл коровы В наличие, Рос...",Говядина
7705,Выгодное Предложение! Курица Цб 82р/кг Наша ко...,Кура
15236,печень говяжья,Говядина
17271,Реализуем охлаждённое мясо свинины Торговый до...,Свинина


Предполагаю, что дубликаты появились, так как по ошибке внесли ещё раз в датафрейм. Также их очень малое количество, удалю их.

In [28]:
df.drop_duplicates(inplace=True)

In [29]:
df.duplicated().sum()

0

Возьму только те виды продукции, для которых в датафрейме есть 500 примеров.

In [30]:
for i in df['mtype'].unique():
  counter = (df['mtype'] == i).sum()
  if counter > 500:
    print(i, counter)

Баранина 1116
Индейка 1339
Говядина 8426
Свинина 3054
Кура 1575
Цыпленок 944


In [31]:
new_df = df.copy()

In [32]:
new_df = new_df.loc[(new_df['mtype'] == 'Баранина') | \
                    (new_df['mtype'] == 'Индейка') | \
                    (new_df['mtype'] == 'Говядина') | \
                    (new_df['mtype'] == 'Свинина') | \
                    (new_df['mtype'] == 'Кура') | \
                    (new_df['mtype'] == 'Цыпленок')]

Сделаю всё строчными буквами.

In [33]:
new_df['text'] = new_df['text'].str.lower()
new_df['mtype'] = new_df['mtype'].str.lower()

## Кодирование категориальных признаков

In [34]:
labelencoder = LabelEncoder()
new_df['mtype'] = labelencoder.fit_transform(new_df['mtype'])

In [35]:
n = 0

for cls in labelencoder.classes_:
  print(n, cls)
  n += 1

0 баранина
1 говядина
2 индейка
3 кура
4 свинина
5 цыпленок


## Деление на выборки

In [65]:
STATE = 2802

In [79]:
features = new_df['text']
target = new_df['mtype']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=STATE)

In [80]:
display(X_train.shape)
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)

(13163,)

(3291,)

(13163,)

(3291,)

## Проверка на длину текста

In [81]:
counter = 0

for txt in X_train:
  if len(txt) > 512:
    counter += 1
print('Amount:', counter)
print('Percent:', counter / len(X_train) * 100)

Amount: 460
Percent: 3.494644078097698


In [82]:
counter = 0

for txt in X_test:
  if len(txt) > 512:
    counter += 1
print('Amount:', counter)
print('Percent:', counter / len(X_test) * 100)

Amount: 131
Percent: 3.980553023397144


Есть процент текстов с более, чем 512 символов. Принимаю решение сократить количество символов этих текстов до максимально возможного значения классификатора, а именно до 512 символов, но сначала посмотрим на эти тексты.

In [83]:
n = 0

for i in X_train:
  if len(i) > 512 and n < 3:
    print(i)
    n += 1

крыло куриное копчено-запеченное небольшой цех по переработке мяса на рынке уже пятый год.

изготавливаем продукты из мяса и птицы методом горячего и холодного копчения

наши клиенты как частные лица и магазины, так и бары и рестораны города. неизменное качество, стабильные поставки, надежное сотрудничество.
работаем только с проверенными поставщиками сырья и специй.
вся продукция производится вручную и подвергается строжайшему контролю качества на всех этапах!
говядина, свинина, баранина, кура, утка, индейка.
лапки лапы ноги павсы chicken paws a+ продаем лапы куриные замороженные, обрезанные категории а+

аттестованные на экспорт во вьетнам. объем-28т.

производим павсы из охлажденного сырья. лапы замораживаются только 1 раз на территории россии, по 15 кг.

a+ это павсы отобранные по весу более 33 грамм

0% льда, 1% лома, 0% черных точек.

поставщик от первого лица (производитель, объявление написал директор производства). склад и отгрузка в санкт-петербурге. возможна доставка до любо

In [84]:
n = 0

for i in X_test:
  if len(i) > 512 and n < 3:
    print(i)
    n += 1

котлеты куриные оптом под стм котлеты куриные собственного производства .
изготавливаем котлеты куриные из филе . рассматриваем запросы по вашему тз.

компания более 20-ти лет поставляет в магазины страны продукцию собственного производства. за эти годы мы остались верны своим принципам: только свежая продукция, только наивысшее качество, и постоянно растущий ассортимент.
производство куриной продукции :
разделка/тушка/ копч.вареные изделия/субпродукты/спец.полуфабрикаты/ тм кудашка для детского питания/тм кур-кума халяль/
гост/ту
говядина быки полутуши охлажденные поставляем мясо, говядина, быки с откорма - 450 + кг.
забой под заказ покупателя. 
бойня - компартмент iv.
полутуши охлажденные - 299 руб./кг. 
цены указаны с ндс.
документы- меркурий, гост.
порядок оплаты – предоплата.
форма оплаты – безналичный платеж, либо оплата в кассу организации.
минимальная отгрузка - 5-ти. тонн.
место отгрузки -  г. владимир, склад бойни.
присутствие представителя заказчика при погрузке обязательно.

In [72]:
X_train = X_train.str.slice(0, 512)

In [73]:
counter = 0

for i in X_train:
  if len(i) > 512:
    counter += 1
print(counter)
print(counter / len(X_train))

0
0.0


In [74]:
X_test = X_test.str.slice(0, 512)

In [76]:
counter = 0

for i in X_test:
  if len(i) > 512:
    counter += 1
print(counter)
print(counter / len(X_test))

0
0.0


## Инициализирую кастомный класс Dataset

In [47]:
class CustomDataset(Dataset):

  def __init__(self, texts, targets, tokenizer, max_len=512):
    self.texts = texts
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    target = self.targets[idx]

    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )

    return {
      'text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

## Инициализация классификатора


In [48]:
class BertClassifier:

  def __init__(self, model_path, tokenizer_path, n_classes=6, epochs=2, model_save_path='/content/bert.pt'):
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model_save_path=model_save_path
        self.max_len = 512
        self.epochs = epochs
        self.out_features = self.model.bert.encoder.layer[1].output.dense.out_features
        self.model.classifier = torch.nn.Linear(self.out_features, n_classes)
        self.model.to(self.device)

  # функция инициализации вспомогательных элементов
  def preparation(self, X_train, y_train, X_test, y_test):
    self.train_set = CustomDataset(X_train, y_train, self.tokenizer)
    self.test_set = CustomDataset(X_test, y_test, self.tokenizer)

    self.train_loader = DataLoader(self.train_set, batch_size=2, shuffle=True)
    self.test_loader = DataLoader(self.test_set, batch_size=2, shuffle=True)

    self.optimizer = AdamW(self.model.parameters(), lr=2e-5, correct_bias=False)
    self.scheduler = get_linear_schedule_with_warmup(self.optimizer,
                                                     num_warmup_steps=0,
                                                     num_training_steps=len(self.train_loader) * self.epochs)
    self.loss_fn = torch.nn.CrossEntropyLoss().to(self.device)

  # обучения для одной эпохи
  def fit(self):
    self.model = self.model.train()
    losses = []
    correct_predictions = 0

    # деление на батчи
    for data in self.train_loader:
        input_ids = data['input_ids'].to(self.device)
        attention_mask = data['attention_mask'].to(self.device)
        targets = data['targets'].to(self.device)

        outputs = self.model(input_ids=input_ids,
                             attention_mask=attention_mask)

        # распределение вероятности по классам и значение ошибки
        preds = torch.argmax(outputs.logits, dim=1)
        loss = self.loss_fn(outputs.logits, targets)

        correct_predictions += torch.sum(preds == targets)

        losses.append(loss.item())

        loss.backward()

        # обрезаем градиенты для предотвращения "взрыва" градиентов
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

        self.optimizer.step()
        self.scheduler.step()
        self.optimizer.zero_grad()

    train_acc = correct_predictions.double() / len(self.train_set)
    train_loss = np.mean(losses)
    return train_acc, train_loss

  # устанавливаю модель в режим оценки
  def eval(self):
    self.model = self.model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for data in self.test_loader:
            input_ids = data['input_ids'].to(self.device)
            attention_mask = data['attention_mask'].to(self.device)
            targets = data['targets'].to(self.device)

            outputs = self.model(input_ids=input_ids,
                                 attention_mask=attention_mask)

            preds = torch.argmax(outputs.logits, dim=1)
            loss = self.loss_fn(outputs.logits, targets)
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    test_acc = correct_predictions.double() / len(self.test_set)
    test_loss = np.mean(losses)
    return test_acc, test_loss

  # функция для обучения на нескольких эпохах
  def train(self):
    best_accuracy = 0
    for epoch in range(self.epochs):
        print(f'Epoch {epoch + 1}/{self.epochs}')
        train_acc, train_loss = self.fit()
        print(f'Train loss {train_loss} accuracy {train_acc}')

        test_acc, test_loss = self.eval()
        print(f'Test loss {test_loss} accuracy {test_acc}')
        print('-' * 10)

        if test_acc > best_accuracy:
            torch.save(self.model, self.model_save_path)
            best_accuracy = test_acc

    self.model = torch.load(self.model_save_path)

  def predict(self, text):
        encoding = self.tokenizer.encode_plus(text,
                                              add_special_tokens=True,
                                              max_length=self.max_len,
                                              return_token_type_ids=False,
                                              truncation=True,
                                              padding='max_length',
                                              return_attention_mask=True,
                                              return_tensors='pt')

        out = {'text': text,
               'input_ids': encoding['input_ids'].flatten(),
               'attention_mask': encoding['attention_mask'].flatten()}

        input_ids = out["input_ids"].to(self.device)
        attention_mask = out["attention_mask"].to(self.device)

        outputs = self.model(input_ids=input_ids.unsqueeze(0),
                             attention_mask=attention_mask.unsqueeze(0))

        prediction = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]

        return prediction

In [49]:
classifier = BertClassifier(model_path='s-nlp/russian_toxicity_classifier',
                            tokenizer_path='s-nlp/russian_toxicity_classifier',
                            n_classes=6,
                            epochs=2,
                            model_save_path='/content/bert.pt')

## Подготовка данных

In [50]:
classifier.preparation(X_train=list(X_train),
                       y_train=list(y_train),
                       X_test=list(X_test),
                       y_test=list(y_test))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Обучение

In [51]:
%%time

classifier.train()

Epoch 1/2
Train loss 0.3957306256366735 accuracy 0.8754083415634734
Test loss 0.19753784399685193 accuracy 0.9574597386812518
----------
Epoch 2/2
Train loss 0.16935712626181526 accuracy 0.9622426498518575
Test loss 0.1940019812651617 accuracy 0.9586751747189304
----------
CPU times: user 41min 4s, sys: 15min 16s, total: 56min 21s
Wall time: 57min 2s


## Тест модели

In [52]:
 %%time

texts = X_test
labels = y_test

predictions = [classifier.predict(txt) for txt in texts]

CPU times: user 1min 58s, sys: 262 ms, total: 1min 58s
Wall time: 1min 59s


In [63]:
precision, recall, f1score = precision_recall_fscore_support(labels, predictions, average='weighted')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.958174056121366, recall: 0.9586751747189304, f1score: 0.9583332118822742
